In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

from itertools import product

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
np.random.seed(123)
torch.manual_seed(123)

### Step 1: Generate Data

In [3]:
from synthetic.generate import SingleTaskTreeDepsGenerator

K = 2
M = 10
N = 10000

# Generate the true class balance to be recovered
class_balance = np.ones(K)/K + np.random.random(K)/5.
class_balance /= class_balance.sum()

data = SingleTaskTreeDepsGenerator(N, M, K, class_balance, edges=[(0,1)])
print (f"LF Dependencies: {data.E}")
print (f"Class Balance: {data.p}")

LF Dependencies: [(0, 1)]
Class Balance: [0.53429356 0.46570644]


In [4]:
from metal.analysis import lf_summary
lf_summary(data.L,data.Y)

,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
0,"[1, 2]",0.8843,0.8843,0.7355,7692,1151,0.869841
1,"[1, 2]",0.9542,0.9542,0.7918,9091,451,0.952735
2,"[1, 2]",0.8087,0.8087,0.6780,6158,1929,0.761469
3,"[1, 2]",0.7613,0.7613,0.6401,5253,2360,0.690004
4,"[1, 2]",0.8327,0.8327,0.6957,6613,1714,0.794164
5,"[1, 2]",0.8550,0.8550,0.7121,7130,1420,0.833918
6,"[1, 2]",0.9227,0.9227,0.7657,8407,820,0.911130
7,"[1, 2]",0.8504,0.8504,0.7091,6968,1536,0.819379
8,"[1, 2]",0.7603,0.7603,0.6417,5242,2361,0.689465
9,"[1, 2]",0.7525,0.7525,0.6392,5120,2405,0.680399


### Step 2: Learn Dependencies using `DependencyLearnerModel`
**TEMP: hardcoding for now**

In [5]:
edges = data.E
L = np.array(data.L.todense())

### Step 3: Recover the class balance using subset of independent LFs

In [6]:
# Generate indices for independent LFs
ind_lfs = []
for i in range(M):
    if i not in list(sum(edges, ())):
        ind_lfs.append(i)
L_ind = L[:,ind_lfs]

In [7]:
%%time
from metal.label_model.class_balance import ClassBalanceModel

cb_model = ClassBalanceModel(K, abstains=True)
cb_model.train_model(L=L_ind, verbose=False)

print(f"Estimated class balance: {cb_model.class_balance}")
print(f"True class balance: {class_balance}")
print()

Estimated class balance: [0.5445712 0.4551824]
True class balance: [0.53429356 0.46570644]

CPU times: user 4.14 s, sys: 148 ms, total: 4.29 s
Wall time: 1.09 s


### Step 4: Train `LabelModel` using $\mu$ from `ClassBalanceModel`

In [8]:
%%time
from metal.label_model import LabelModel
label_model = LabelModel(verbose=True,k=K, seed=123)

label_model.train_model(
    data.L, 
    cond_probs = cb_model.cond_probs, 
    ind_lfs = ind_lfs, 
    deps = edges, 
    lr = 1e-3, 
    n_epochs = 500, 
    log_train_every = 100
)

print()
for y in range(K):
    mu_idx = [il*K + y for il in ind_lfs]
    err = np.linalg.norm(label_model.mu[mu_idx,:].detach().numpy() - cb_model.cond_probs[:,y+1,:])/(M*1.0)
    print(f"Error between mu and mu_init for y = {y}: {err}")
print()

Computing O^{-1}...
Estimating Z...
[100 epo]: TRAIN:[loss=12.277]
[200 epo]: TRAIN:[loss=12.142]
[300 epo]: TRAIN:[loss=12.141]
[400 epo]: TRAIN:[loss=12.141]
[500 epo]: TRAIN:[loss=12.141]
Finished Training
Estimating \mu...
[100 epo]: TRAIN:[loss=0.008]
[200 epo]: TRAIN:[loss=0.003]
[300 epo]: TRAIN:[loss=0.002]
[400 epo]: TRAIN:[loss=0.001]
[500 epo]: TRAIN:[loss=0.000]
Finished Training

Error between mu and mu_init for y = 0: 0.008222036063671112
Error between mu and mu_init for y = 1: 0.0076375149190425875

CPU times: user 13.2 s, sys: 2.96 s, total: 16.2 s
Wall time: 847 ms


In [9]:
print('Trained Label Model Metrics:')
scores = label_model.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Trained Label Model Metrics:
Accuracy: 0.985
Precision: 0.986
Recall: 0.986
F1: 0.986
        y=1    y=2   
 l=1   5314    75    
 l=2    76    4535   


### Step 4b: Compare to MajorityVote and No Class Balance+Dependencies LabelModel

**Baseline: No Class Balance or Dependencies**

In [10]:
%%time
from metal.label_model import LabelModel
label_model = LabelModel(k=K, seed=123)

label_model.train_model(data.L, lr=1e-3, n_epochs=500, log_train_every=50)

Computing O...
Estimating \mu...
[50 epo]: TRAIN:[loss=0.107]
[100 epo]: TRAIN:[loss=0.012]
[150 epo]: TRAIN:[loss=0.007]
[200 epo]: TRAIN:[loss=0.006]
[250 epo]: TRAIN:[loss=0.006]
[300 epo]: TRAIN:[loss=0.005]
[350 epo]: TRAIN:[loss=0.005]
[400 epo]: TRAIN:[loss=0.004]
[450 epo]: TRAIN:[loss=0.004]
[500 epo]: TRAIN:[loss=0.004]
Finished Training
CPU times: user 14.2 s, sys: 124 ms, total: 14.3 s
Wall time: 528 ms


In [11]:
print('Trained Label Model Metrics:')
scores = label_model.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Trained Label Model Metrics:
Accuracy: 0.982
Precision: 0.984
Recall: 0.983
F1: 0.984
        y=1    y=2   
 l=1   5297    84    
 l=2    93    4526   


**Baseline: Majority Vote**

In [12]:
from metal.label_model.baselines import MajorityLabelVoter

mv = MajorityLabelVoter(k=K,seed=123)
print('Majority Label Voter Metrics:')
scores = mv.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Majority Label Voter Metrics:
Accuracy: 0.978
Precision: 0.971
Recall: 0.988
F1: 0.979
        y=1    y=2   
 l=1   5323    157   
 l=2    67    4453   
